# Model Problem on the Unit Disk

Let $\Omega$ be the **unit disk** in $\mathbb{R}^2$.

We want to solve the model problem

$$
- \Delta u = f \quad \text{in } \Omega, \qquad u = u_D \quad \text{on } \partial \Omega,
$$

where $\Omega$ is embedded in a **background domain** $(-1.25,\ 1.25)^2$.

We consider the Dirichlet problem with prescribed boundary data $u_D$ and a right-hand side function $f$.



In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *

In [2]:
maxh = 0.1
# Finite element space order
order = 1
# Stabilization parameter for ghost-penalty
gamma_stab = 0.1
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order

In [3]:
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
r = sqrt(x**2 + y**2)
levelset = r-1

exact = sin(x+y)
coeff_f = - (exact.Diff(x).Diff(x) + exact.Diff(y).Diff(y)).Compile()

In [14]:
maxh = 0.1
gamma_stab = 0.1
beta0 =1
beta2 = 1
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)

lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)
#cut_elements = ci.GetElementsOfType(HASIF)  


interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
#interface_facet_set |= GetFacetsWithNeighborTypes(mesh, a=any, b=hasneg)


h = specialcf.mesh_size
n = Normalize(grad(lsetp1))

# integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)
dw_int = dFacetPatch(definedonelements=interior_facets)
dw_interface = dFacetPatch(definedonelements=interface_facet_set)

#ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
V = VectorH1(mesh, order=1,dgjumps = True)
Q = H1(mesh, order=0)
Z = NumberSpace(mesh)
X = V*Q*Z
(u,p,z),(v,q,y) = X.TnT()
gfu = GridFunction(X)

def jump(f):
    return f - f.Other()
def jumpn(f):
    return f*n - f.Other()*n

stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
a = BilinearForm(stokes)
#a += gamma_stab / h**2 * (grad(u) - grad(u.Other())) * (grad(v) - grad(v.Other())) * dw
a += -beta0 * h * jump(p) * jump(q) * dw_int # pressure stabilization
a += beta2*h* jumpn(u) * jumpn(v) * dw_interface #velocity ghost penalty
a += - beta0 * h * jump(p) * jump(q) * dw_interface #pressure ghost penalty
a += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds 
a += p*y *dx + q *z*dx

a.Assemble()

f = CF((1,0))
rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
rhs += InnerProduct(f, v)* dx
rhs.Assemble()

gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec

#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
Draw(gfu.components[0], mesh, "u")
Draw(gfu.components[1], mesh, "p")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [19]:
maxh = 0.1
gamma_stab = 0.1
beta0 =1
beta1 = 1
beta2 = 1
beta3 = 1
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)

lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)
#cut_elements = ci.GetElementsOfType(HASIF)  


interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
#interface_facet_set |= GetFacetsWithNeighborTypes(mesh, a=any, b=hasneg)


h = specialcf.mesh_size
n = Normalize(grad(lsetp1))

# integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)
dw_int = dFacetPatch(definedonelements=interior_facets)
dw_interface = dFacetPatch(definedonelements=interface_facet_set)

#ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
V = VectorH1(mesh, order=1,dgjumps = True)
Q = H1(mesh, order=1)
Z = NumberSpace(mesh)
X = V*Q*Z
(u,p,z),(v,q,y) = X.TnT()
gfu = GridFunction(X)

def jump(f):
    return f - f.Other()
def jumpn(f):
    return f*n - f.Other()*n

stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
a = BilinearForm(stokes)
#a += gamma_stab / h**2 * (grad(u) - grad(u.Other())) * (grad(v) - grad(v.Other())) * dw
a += -beta0 * h * jump(p) * jump(q) * dw_int # pressure stabilization
a += beta2*h* jumpn(u) * jumpn(v) * dw_interface #velocity ghost penalty
a += beta3 *h**3 *(Grad(p)*n-Grad(p.Other())*n)*(Grad(q)*n-Grad(q.Other())*n) * dw_interface #pressure ghost penalty
a += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds 
a += -beta1 * h**2 * Grad(p)  * Grad(q) * dx #cterm 
a += n*v* p * ds + n*u*q * ds #b
a += p*y *dx + q *z*dx

a.Assemble()

f = CF((0,x))
g = CoefficientFunction((0, 0))
rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
rhs += InnerProduct(f, v)* dx
rhs += - beta1 * h**2 * f* Grad(q) * dx
rhs += g * (gamma_stab / h * v - Grad(v)*n + q*n) * ds
rhs.Assemble()

gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec

#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
Draw(gfu.components[0], mesh, "u")
Draw(gfu.components[1], mesh, "p")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene